# Mutlinomial Logistic Regression




In [11]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%matplotlib inline
from pathlib import Path
import pickle
from collections import OrderedDict
import pandas as pd
import numpy as np
from scipy import stats

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import (roc_curve, roc_auc_score, confusion_matrix, accuracy_score, f1_score, 
                             precision_recall_curve) 
from mlxtend.plotting import plot_confusion_matrix

import theano
import pymc3 as pm
from pymc3.variational.callbacks import CheckParametersConvergence
import statsmodels.formula.api as smf

import arviz as az
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import seaborn as sns
from IPython.display import HTML
data_path = Path('data')
fig_path = Path('figures')
model_path = Path('models')
for p in [data_path, fig_path, model_path]:
    if not p.exists():
        p.mkdir()

In [14]:
!pip install git+https://github.com/pymc-devs/pymc3
!pip install git+git://github.com/arviz-devs/arviz.git



  Cloning https://github.com/pymc-devs/pymc3 to /private/var/folders/kd/p1z7_f_974d7tkw1zmvskn_m0000gn/T/pip-req-build-2v2o15ym
  Running command git clone --filter=blob:none -q https://github.com/pymc-devs/pymc3 /private/var/folders/kd/p1z7_f_974d7tkw1zmvskn_m0000gn/T/pip-req-build-2v2o15ym
  error: RPC failed; curl 56 LibreSSL SSL_read: Connection reset by peer, errno 54
  error: 4131 bytes of body are still expected
  fetch-pack: unexpected disconnect while reading sideband packet
  fatal: early EOF
  fatal: index-pack failed
  Auto packing the repository in background for optimum performance.
  See "git help gc" for manual housekeeping.
  Auto packing the repository in background for optimum performance.
  See "git help gc" for manual housekeeping.
  Auto packing the repository in background for optimum performance.
  See "git help gc" for manual housekeeping.
  Auto packing the repository in background for optimum performance.
  See "git help gc" for manual housekeeping.
  Auto pa

In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
%config InlineBackend.figure_format = 'retina'
# Initialize random number generator
RANDOM_SEED = 8927
np.random.seed(RANDOM_SEED)
az.style.use("arviz-darkgrid")
# True parameter values
alpha, sigma = 1, 1
beta = [1, 2.5]

# Size of dataset
size = 100

# Predictor variable
X1 = np.random.randn(size)
X2 = np.random.randn(size) * 0.2

# Simulate outcome variable
Y = alpha + beta[0] * X1 + beta[1] * X2 + np.random.randn(size) * sigma

In [ ]:
fig, axes = plt.subplots(1, 2, sharex=True, figsize=(10, 4))
axes[0].scatter(X1, Y, alpha=0.6)
axes[1].scatter(X2, Y, alpha=0.6)
axes[0].set_ylabel("Y")
axes[0].set_xlabel("X1")
axes[1].set_xlabel("X2");

In [ ]:
data = pd.read_csv('bank.csv')
data.info()

categorical_features = data.select_dtypes(include=[np.object])

numeric_features = data.select_dtypes(include=[np.number])

cols = numeric_features.columns
numeric_features.loc[:, cols] = preprocessing.scale(numeric_features.loc[:, cols])

le = preprocessing.LabelEncoder()

categorical_features = data[categorical_features.columns].apply(lambda x: le.fit_transform(x))

data = pd.concat([categorical_features, numeric_features], axis=1)
data.info()